# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "clean_city_data.csv"

city_data_df = pd.read_csv(city_data)

city_data_df = city_data_df.drop(columns=['Unnamed: 0'])
city_data_df['City'] = city_data_df['City'].str.capitalize()
city_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Faya,0,SA,1607393704,66,18.39,42.45,51.80,0.74
1,Rikitea,6,PF,1607393705,81,-23.12,-134.97,75.60,13.91
2,Carnarvon,0,AU,1607393705,65,-24.87,113.63,80.60,9.17
3,Matara,0,LK,1607393705,82,5.95,80.54,79.00,3.76
4,Broome,90,US,1607393706,79,42.25,-75.83,27.00,12.75
...,...,...,...,...,...,...,...,...,...
553,San jose,100,ML,1607393703,29,13.30,-4.90,73.51,9.26
554,Luderitz,81,NaN,1607393807,92,-26.65,15.16,60.67,3.47
555,Krasnoselkup,81,RU,1607393807,88,65.70,82.47,-11.52,18.57
556,Asosa,82,ET,1607393807,40,10.07,34.53,67.89,4.56


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = pd.DataFrame(city_data_df, columns=['City', 'Max Temp', 'Wind Speed', 'Cloudiness'])

max_temp = (ideal_weather['Max Temp']) <= 102 & (ideal_weather['Max Temp'] > 45)
wind_speed = ideal_weather['Wind Speed'] < 15
cloudiness = ideal_weather['Cloudiness'] < 10

ideal_weather[max_temp & wind_speed & cloudiness]
ideal_weather

,City,Max Temp,Wind Speed,Cloudiness
0,Faya,51.80,0.74,0
1,Rikitea,75.60,13.91,6
2,Carnarvon,80.60,9.17,0
3,Matara,79.00,3.76,0
4,Broome,27.00,12.75,90
...,...,...,...,...
553,San jose,73.51,9.26,100
554,Luderitz,60.67,3.47,81
555,Krasnoselkup,-11.52,18.57,81
556,Asosa,67.89,4.56,82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(city_data_df, columns=['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Faya,SA,18.39,42.45,
1,Rikitea,PF,-23.12,-134.97,
2,Carnarvon,AU,-24.87,113.63,
3,Matara,LK,5.95,80.54,
4,Broome,US,42.25,-75.83,
...,...,...,...,...,...
553,San jose,ML,13.30,-4.90,
554,Luderitz,NaN,-26.65,15.16,
555,Krasnoselkup,RU,65.70,82.47,
556,Asosa,ET,10.07,34.53,


In [8]:
#coordinates_lat_lng = []
#target_city = hotel_df['City']
#target_type = 'hotel'
#hotel_df['Coordinates'] = hotel_df{'Lat'} + " , " + hotel_df{'Lng'}
#radius = 5000
#params = {'location': coordinates_lat_lng,
#          'types': target_type,
#          'radius': radius,
#          'key': g_key}

#base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#response = requests.get(base_url, params)
#print(response.url)

#hotel_distance = response.json()
#print(json.dumps(hotel_distance, indent=2, sort_keys=True))

target_coordinates = f"{hotel_df['Lat'][0]}, {hotel_df['Lng'][0]}"
target_search = 'hotel'
target_radius = 5000
target_type = 'hotel'

params = {
            'location': target_coordinates,
            'keyworkd': target_search,
            'radius': target_radius,
            'type': target_type,
            'key': g_key
}

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

response = requests.get(base_url, params=params)
pprint(response.json(), depth=1)

{'html_attributions': [],
 'next_page_token': 'ATtYBwIqfj8CKahHssIG1jPVZZQT4mfXY51-T62AoE3WT_mZup21S559VLUbDw8D7_fHkuEvu2HwDukny81ODX2YgdmK_idi2EcpyfYTrRnCzKV7jDp83-g9m_0U2wO0OqCK0GmjVvzgriwqTeEfsnu-wt24l6MSuTh1S8b2TTy_D7Ptdb3Md5yHcEq8sIJChQjC-2I6MyfZ3Mua3LMgD2jqyqQSCUREC-1ids3PLKXtHcZHRAgDwuT0595xCOsxsVsORu9Wp-1YbCI0dAyG4yX70hSWiuFWeacTXBhmGbND4-tb0oV2msYNCp4JKpHMLOiSOxAn3_O65YatoMh9Plk8pHODOTi_Z_8LzOaY5LpvmNRZwaMNYyxeJLj97lpiAKKNIu4cpcIxNO_F6JZ-3o-wzJ6xuYuxSiIWcRbpPm8DhMtqwbdG_phR7w',
 'results': [...],
 'status': 'OK'}


In [9]:
#1st Hotel
hotel_df.iloc[0][0]

'Faya'

In [ ]:
hotel_name = []

params = {
            'radius': 5000,
            'types': 'hotel',
            'keyword': 'hotel',
            'key': g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    try:
        hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [ ]:
hotel_name

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig.show()